In [1]:
import json
import glob
import random
import numpy as np
import pandas as pd
import fastai
from fastai import *
from fastai.vision import *
from fastai.vision.image import *
from fastai.vision.data import  * 
from fastai.callbacks.tracker import *
from torch.utils.data import Sampler, BatchSampler, SequentialSampler
from PIL import Image as IMG
from PIL import ImageFile
import matplotlib.pyplot as plt
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
images = glob.glob('/data/farfetch-scrape-v2/images/*.jpg')
imgids = [os.path.basename(fn).split('.')[0] for fn in images]
test_df = pd.DataFrame({'imgids': imgids})
test_imagelist = ImageList.from_df(df=test_df, path='/data/farfetch-scrape-v2/images/', suffix='.jpg')

In [3]:
model_stage1 = load_learner('/data/logs/imaterialist-fastai/', 'imaterialist-stage1-40epochs.pkl', test=test_imagelist)


In [4]:
features = []
def get_features(module, input, output) :
    features.append(output)
handle = model_stage1.model[1][4].register_forward_hook(get_features)

In [5]:
preds, y = model_stage1.get_preds(ds_type=DatasetType.Test)

OSError: Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/maskrcnn/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/miniconda3/envs/maskrcnn/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/ubuntu/miniconda3/envs/maskrcnn/lib/python3.7/site-packages/fastai/data_block.py", line 636, in __getitem__
    if self.item is None: x,y = self.x[idxs],self.y[idxs]
  File "/home/ubuntu/miniconda3/envs/maskrcnn/lib/python3.7/site-packages/fastai/data_block.py", line 109, in __getitem__
    if isinstance(idxs, Integral): return self.get(idxs)
  File "/home/ubuntu/miniconda3/envs/maskrcnn/lib/python3.7/site-packages/fastai/vision/data.py", line 270, in get
    res = self.open(fn)
  File "/home/ubuntu/miniconda3/envs/maskrcnn/lib/python3.7/site-p

In [8]:
masked_look_features = [feat.cpu().numpy() for feat in features]
masked_look_features = np.vstack(masked_look_features)

In [9]:
images = glob.glob('/data/runway2shop/products/*.jpg')
imgids = [os.path.basename(fn).split('.')[0] for fn in images]
test_df = pd.DataFrame({'imgids': imgids})
test_imagelist = ImageList.from_df(df=test_df, path='/data/runway2shop/products/', suffix='.jpg')

model_stage1 = load_learner('/data/logs/imaterialist-fastai/', 'imaterialist-stage1-40epochs.pkl', test=test_imagelist)

features = []
def get_features(module, input, output) :
    features.append(output)
handle = model_stage1.model[1][4].register_forward_hook(get_features)

preds, y = model_stage1.get_preds(ds_type=DatasetType.Test)

In [10]:
product_features = [feat.cpu().numpy() for feat in features]
product_features = np.vstack(product_features)

In [12]:
masked_look_features.shape

(455, 512)

In [ ]:
np.save('masked_look_imaterialist_features.npy')